In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import mpmath as mp
import numpy as np
import scipy as sc
from mpl_toolkits import mplot3d
from array import *
from scipy.integrate import *
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from numpy.random import normal
from numpy import hstack

from statsmodels.distributions.empirical_distribution import ECDF
from matplotlib.ticker import FuncFormatter, MultipleLocator

!pip install pymannkendall
import pymannkendall as mk

plt.rcParams['figure.figsize'] = [5, 5]
plt.rcParams['figure.dpi'] = 100
# plt.rcParams['figure.figsize'] = [10.24, 7.6]
# plt.rcParams['figure.dpi'] = 300

In [2]:
naon_avg = pd.read_csv("naon_avg.csv")
naop_avg = pd.read_csv("naop_avg.csv")

In [3]:
naon_avg

,yymo,Unnamed: 0,Unnamed: 0.1,sno,yy,mo,dd,total_wl,precipitation,streamflow,...,emp_skewv,nao_yymmo,nao_index,twl_max,prec_max,intensity,nint_twl_pcp,nint_twl_sflow,nint_sflow_ssurge,nint_pcp_ssurg
0,196501,0.0,5.000000,6.000000,1965.0,1.0,6.000000,1.610000,6.530000,60.300000,...,0.679268,196501.0,-0.95,1.7500,15.400000,1.705215,1.705215,1.531748,1.277509,1.450975
1,196502,1.5,51.000000,52.000000,1965.0,2.0,21.000000,1.550000,1.730000,9.430000,...,0.661609,196502.0,-0.61,1.6850,23.100000,1.485656,1.485656,0.935930,0.701454,1.251179
2,196601,6.0,372.333333,373.333333,1966.0,1.0,8.333333,1.873333,11.296667,17.900000,...,0.970844,196601.0,-0.67,1.7800,52.600000,1.833100,1.833100,1.162905,1.148179,1.818373
3,196602,9.5,407.750000,408.750000,1966.0,2.0,12.750000,1.687500,6.350000,23.425000,...,0.871797,196602.0,-0.43,1.7900,10.015000,1.615629,1.615629,1.211000,1.122992,1.527620
4,196612,12.0,714.000000,715.000000,1966.0,12.0,16.000000,1.510000,1.730000,30.600000,...,0.827744,196612.0,-0.21,1.6600,43.800000,1.451294,1.451294,1.208785,1.174806,1.417315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,201104,771.5,15423.000000,15424.000000,2011.0,4.0,3.000000,1.669000,4.450000,24.700000,...,0.995665,201104.0,-1.02,1.6995,24.900000,1.672957,1.672957,1.229759,1.269476,1.712674
138,201109,773.0,15582.000000,15583.000000,2011.0,9.0,11.000000,1.315000,0.000000,88.400000,...,0.487377,201109.0,-2.50,1.7300,0.300000,0.808747,0.808747,1.296251,1.211462,0.723958
139,201110,776.0,15628.000000,15629.000000,2011.0,10.0,29.000000,1.814800,10.400000,80.740000,...,0.938812,201110.0,-1.92,1.8164,40.820000,1.722734,1.722734,1.666365,1.622287,1.678656
140,201111,780.0,15651.666667,15652.666667,2011.0,11.0,27.666667,1.650000,2.110000,48.866667,...,0.769540,201111.0,-2.95,1.6760,20.873333,1.528390,1.528390,1.463428,1.286221,1.351184


In [4]:
naop_avg

,yymo,Unnamed: 0,Unnamed: 0.1,sno,yy,mo,dd,total_wl,precipitation,streamflow,...,emp_skewv,nao_yymmo,nao_index,twl_max,prec_max,intensity,pint_twl_pcp,pint_twl_sflow,pint_sflow_ssurge,pint_pcp_ssurg
0,196512,3.5,353.000000,354.000000,1965.0,12.0,20.000000,1.600000,2.585000,24.850000,...,0.855285,196512.0,0.25,1.665000,8.545000,1.560787,1.560787,1.188767,1.122721,1.494741
1,196811,31.0,1422.333333,1423.333333,1968.0,11.0,23.333333,1.623333,0.173333,23.500000,...,0.562242,196811.0,0.03,1.790000,15.543333,1.231077,1.231077,1.179013,0.820604,0.872668
2,196910,38.0,1748.000000,1749.000000,1969.0,10.0,15.000000,1.530000,0.720000,43.000000,...,0.543032,196910.0,0.95,1.700000,5.840000,1.370968,1.370968,1.349484,1.013706,1.035191
3,197001,40.5,1841.500000,1842.500000,1970.0,1.0,16.500000,1.720000,2.830000,34.925000,...,0.825625,197001.0,1.06,1.757500,22.700000,1.475504,1.475504,1.359253,1.214698,1.330948
4,197003,43.0,1893.000000,1894.000000,1970.0,3.0,9.000000,1.580000,4.880000,22.300000,...,0.774576,197003.0,1.20,1.660000,8.620000,1.645608,1.645608,1.156318,1.015683,1.504973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,200601,645.0,13934.333333,13935.333333,2006.0,1.0,8.333333,1.730667,0.140000,74.233333,...,0.884015,200601.0,0.54,1.706667,8.110000,1.255897,1.255897,1.644290,1.553615,1.165222
91,200602,647.0,13959.000000,13960.000000,2006.0,2.0,2.000000,1.658000,19.300000,25.700000,...,0.916741,200602.0,0.38,1.658000,66.400000,1.875940,1.875940,1.237537,1.202856,1.841260
92,200909,682.0,14870.000000,14871.000000,2009.0,9.0,19.000000,1.658000,3.400000,56.300000,...,0.957542,200909.0,0.33,1.658000,31.700000,1.630626,1.630626,1.523078,1.529198,1.636746
93,201001,707.5,14984.200000,14985.200000,2010.0,1.0,13.200000,1.814300,3.987000,37.560000,...,0.874589,201001.0,0.05,1.778200,13.977000,1.558626,1.558626,1.358237,1.246946,1.447335


In [5]:
naop_month=naop_avg.groupby(naop_avg.mo)


In [6]:
na=naop_avg.sort_values(['mo'], ascending=[True])
na

,yymo,Unnamed: 0,Unnamed: 0.1,sno,yy,mo,dd,total_wl,precipitation,streamflow,...,emp_skewv,nao_yymmo,nao_index,twl_max,prec_max,intensity,pint_twl_pcp,pint_twl_sflow,pint_sflow_ssurge,pint_pcp_ssurg
62,199601,400.5,10355.300000,10356.300000,1996.0,1.0,8.300000,1.749000,2.10500,36.810000,...,0.899152,199601.0,1.01,1.79800,14.804000,1.414140,1.414140,1.344078,1.274997,1.345059
42,199201,322.5,8905.900000,8906.900000,1992.0,1.0,17.900000,1.687000,2.33100,16.430000,...,0.796188,199201.0,0.08,1.72800,7.875000,1.474493,1.474493,1.111705,0.947220,1.310009
31,198601,217.0,7134.000000,7135.000000,1986.0,1.0,11.000000,1.690000,0.21000,26.300000,...,0.849484,198601.0,1.04,1.69000,0.990000,1.268775,1.268775,1.260487,1.144715,1.153003
93,201001,707.5,14984.200000,14985.200000,2010.0,1.0,13.200000,1.814300,3.98700,37.560000,...,0.874589,201001.0,0.05,1.77820,13.977000,1.558626,1.558626,1.358237,1.246946,1.447335
53,199401,354.0,9638.333333,9639.333333,1994.0,1.0,19.333333,1.663333,0.60000,35.733333,...,0.827511,199401.0,0.85,1.78000,14.893333,1.291853,1.291853,1.293234,1.220218,1.218836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,199212,338.5,9249.500000,9250.500000,1992.0,12.0,26.500000,1.600000,6.67000,31.550000,...,0.596232,199212.0,0.33,1.79500,20.850000,1.614593,1.614593,1.268870,0.951613,1.297335
30,198512,212.5,7103.250000,7104.250000,1985.0,12.0,11.250000,1.695000,3.53625,24.400000,...,0.865374,198512.0,0.33,1.77125,9.172500,1.513037,1.513037,1.214475,1.134061,1.432623
9,197612,113.5,4351.500000,4352.500000,1976.0,12.0,14.500000,1.635000,0.69000,56.250000,...,0.859843,197612.0,1.20,1.77500,9.310000,1.409505,1.409505,1.480938,1.416135,1.344702
7,197412,94.0,3645.666667,3646.666667,1974.0,12.0,25.666667,1.710000,3.30000,32.300000,...,0.699520,197412.0,0.27,1.71000,11.440000,1.554465,1.554465,1.291980,1.021144,1.283629


In [7]:
n_jan_mar = naon_avg[naon_avg['mo'] < 4]
n_apr_jun = naon_avg[(naon_avg['mo'] >= 4) & (naon_avg['mo'] < 7)]
n_jul_sep = naon_avg[(naon_avg['mo'] >= 7) & (naon_avg['mo'] < 10)]
n_oct_dec = naon_avg[(naon_avg['mo'] >= 10) & (naon_avg['mo'] <= 12)]
n_oct_dec
#n_jan_mar 

,yymo,Unnamed: 0,Unnamed: 0.1,sno,yy,mo,dd,total_wl,precipitation,streamflow,...,emp_skewv,nao_yymmo,nao_index,twl_max,prec_max,intensity,nint_twl_pcp,nint_twl_sflow,nint_sflow_ssurge,nint_pcp_ssurg
4,196612,12.0,714.000000,715.000000,1966.0,12.0,16.000000,1.510000,1.730000,30.600000,...,0.827744,196612.0,-0.21,1.660000,43.800000,1.451294,1.451294,1.208785,1.174806,1.417315
7,196710,18.0,1007.666667,1008.666667,1967.0,10.0,5.666667,1.736667,0.270000,53.700000,...,0.763228,196710.0,-0.06,1.760000,7.130000,1.325194,1.325194,1.509031,1.313528,1.129691
8,196712,20.0,1068.000000,1069.000000,1967.0,12.0,5.000000,1.460000,12.800000,34.000000,...,0.321051,196712.0,-0.68,1.690000,1.240000,1.677929,1.677929,1.188576,0.703876,1.193230
13,196810,29.0,1392.000000,1393.000000,1968.0,10.0,24.000000,1.620000,0.510000,74.500000,...,0.500574,196810.0,-0.18,1.770000,6.230000,1.390284,1.390284,1.612074,1.173594,0.951804
14,196812,33.5,1442.500000,1443.500000,1968.0,12.0,13.500000,1.670000,3.790000,83.750000,...,0.622848,196812.0,-0.29,1.670000,15.240000,1.579179,1.579179,1.630977,1.295741,1.243944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,201011,734.5,15294.166667,15295.166667,2010.0,11.0,24.166667,1.742000,1.676667,37.133333,...,0.922659,201011.0,-1.57,1.778667,6.500000,1.496929,1.496929,1.352947,1.335033,1.479015
133,201012,740.5,15322.166667,15323.166667,2010.0,12.0,22.166667,1.795500,10.473333,134.383333,...,0.885429,201012.0,-2.04,1.852000,26.916667,1.777594,1.777594,1.782789,1.696938,1.691742
139,201110,776.0,15628.000000,15629.000000,2011.0,10.0,29.000000,1.814800,10.400000,80.740000,...,0.938812,201110.0,-1.92,1.816400,40.820000,1.722734,1.722734,1.666365,1.622287,1.678656
140,201111,780.0,15651.666667,15652.666667,2011.0,11.0,27.666667,1.650000,2.110000,48.866667,...,0.769540,201111.0,-2.95,1.676000,20.873333,1.528390,1.528390,1.463428,1.286221,1.351184


In [8]:
p_jan_mar = naop_avg[naop_avg['mo'] < 4]
p_apr_jun = naop_avg[(naop_avg['mo'] >= 4) & (naop_avg['mo'] < 7)]
p_jul_sep = naop_avg[(naop_avg['mo'] >= 7) & (naop_avg['mo'] < 10)]
p_oct_dec = naop_avg[(naop_avg['mo'] >= 10) & (naop_avg['mo'] <= 12)]
p_oct_dec

,yymo,Unnamed: 0,Unnamed: 0.1,sno,yy,mo,dd,total_wl,precipitation,streamflow,...,emp_skewv,nao_yymmo,nao_index,twl_max,prec_max,intensity,pint_twl_pcp,pint_twl_sflow,pint_sflow_ssurge,pint_pcp_ssurg
0,196512,3.5,353.000000,354.000000,1965.0,12.0,20.000000,1.600000,2.585000,24.850000,...,0.855285,196512.0,0.25,1.665000,8.545000,1.560787,1.560787,1.188767,1.122721,1.494741
1,196811,31.0,1422.333333,1423.333333,1968.0,11.0,23.333333,1.623333,0.173333,23.500000,...,0.562242,196811.0,0.03,1.790000,15.543333,1.231077,1.231077,1.179013,0.820604,0.872668
2,196910,38.0,1748.000000,1749.000000,1969.0,10.0,15.000000,1.530000,0.720000,43.000000,...,0.543032,196910.0,0.95,1.700000,5.840000,1.370968,1.370968,1.349484,1.013706,1.035191
6,197411,91.5,3592.500000,3593.500000,1974.0,11.0,2.500000,1.710000,0.000000,163.500000,...,0.972396,197411.0,0.61,1.710000,6.400000,1.206809,1.206809,1.829561,1.831729,1.208976
7,197412,94.0,3645.666667,3646.666667,1974.0,12.0,25.666667,1.710000,3.300000,32.300000,...,0.699520,197412.0,0.27,1.710000,11.440000,1.554465,1.554465,1.291980,1.021144,1.283629
8,197611,111.0,4328.333333,4329.333333,1976.0,11.0,21.333333,1.706667,2.953333,66.733333,...,0.736134,197611.0,1.33,1.706667,10.780000,1.521123,1.521123,1.596328,1.364784,1.289579
9,197612,113.5,4351.500000,4352.500000,1976.0,12.0,14.500000,1.635000,0.690000,56.250000,...,0.859843,197612.0,1.20,1.775000,9.310000,1.409505,1.409505,1.480938,1.416135,1.344702
13,197810,129.0,4944.000000,4945.000000,1978.0,10.0,4.000000,1.660000,0.750000,14.900000,...,0.907943,197810.0,0.07,1.660000,13.700000,1.457924,1.457924,1.082876,1.036083,1.411131
16,198011,150.0,5632.400000,5633.400000,1980.0,11.0,19.400000,1.604000,4.278000,183.560000,...,0.873645,198011.0,0.71,1.726000,31.112000,1.553882,1.553882,1.691649,1.698100,1.560334
17,198111,153.5,5996.500000,5997.500000,1981.0,11.0,27.500000,1.690000,2.070000,47.650000,...,0.875526,198111.0,0.54,1.690000,40.400000,1.562444,1.562444,1.472715,1.384292,1.474021


In [9]:
n_jan_mar.to_csv('n_jan_mar.csv')
n_apr_jun.to_csv('n_apr_jun.csv')
n_jul_sep.to_csv('n_jul_sep.csv')
n_oct_dec.to_csv('n_oct_dec.csv')

In [10]:
p_jan_mar.to_csv('p_jan_mar.csv')
p_apr_jun.to_csv('p_apr_jun.csv')
p_jul_sep.to_csv('p_jul_sep.csv')
p_oct_dec.to_csv('p_oct_dec.csv')